Import packages

In [1]:
import pandas as pds
import datetime as dt

Import data

In [2]:
all_data = pds.read_csv('../../bus203_all.csv')
df = all_data.head(3464837)

In [3]:
df['timestamp'] = pds.to_datetime(df['timestamp']).dt.tz_localize("UTC").dt.tz_convert("Europe/Stockholm")

/home/elias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Extract labels

In [4]:
df['time_since_journey_start'] = -1
df['label'] = -1
segment_time_start = df.iloc[0]['timestamp']
segment = df.iloc[0]['segment_number']
segment_start_index = 0

journey = df.iloc[0]['journey_number']
journey_time = 0

for index, row in df.iterrows():
    
    if df.loc[index, 'segment_number'] != segment or index == len(df.index)-1:
        segment_time_end = df.iloc[index-1]['timestamp']
        
        # Total time for segment
        timediff = (segment_time_end-segment_time_start).total_seconds()
        
        # Update time since journey start
        journey_time = journey_time + timediff
        
        # Update labels and time since journey start for the observations in the segment
        for i in range(segment_start_index, index):
            df.at[i, 'label'] = timediff
            
        # Don't miss the last observation
        if index == len(df.index)-1:
            df.at[index, 'label'] = timediff
        
        # Update initial values for the next segment
        segment_time_start = df.iloc[index]['timestamp']
        segment = df.iloc[index]['segment_number']
        segment_start_index = index
        
        if df.iloc[index]['journey_number'] != journey: # New journey, reset time since journey start
            journey_time = 0
            journey = df.iloc[index]['journey_number']
    # Set time travelled since journey started
    df.at[index,'time_since_journey_start'] = journey_time
            
df.to_csv("bus203_all_labelled_2.csv")

/home/elias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/elias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
